In [ ]:
# ============================================
# 0. Import libraries & Setup
# ============================================
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.linear_model import Ridge
from sklearn.preprocessing import RobustScaler
import torch
import torch.nn as nn
import torch.optim as optim
import warnings
import time
import random
from tqdm import tqdm

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(42)
warnings.filterwarnings('ignore')

# ---- 全局参数 ----
TEST_DAYS = 180
MIN_INVESTMENT = 0
MAX_INVESTMENT = 2
TARGET_VOL = 0.23

# [核心评分工具]
def calc_official_metric(strat_ret, mkt_ret, rf, trading_days=252):
    strat_ret = np.nan_to_num(np.array(strat_ret))
    mkt_ret = np.nan_to_num(np.array(mkt_ret))
    rf = np.nan_to_num(np.array(rf))
    strat_excess = strat_ret - rf
    strat_mean = np.mean(strat_excess)
    strat_std = np.std(strat_ret)
    if strat_std == 0: return 0.0
    sharpe = (strat_mean / strat_std) * np.sqrt(trading_days)
    strat_vol = strat_std * np.sqrt(trading_days) * 100
    mkt_excess = mkt_ret - rf
    mkt_mean = np.mean(mkt_excess)
    mkt_std = np.std(mkt_ret)
    mkt_vol = mkt_std * np.sqrt(trading_days) * 100
    if mkt_vol == 0: excess_vol = 0
    else: excess_vol = max(0, strat_vol / mkt_vol - 1.2)
    vol_penalty = 1 + excess_vol
    ann_strat_mean = strat_mean * trading_days * 100
    ann_mkt_mean = mkt_mean * trading_days * 100
    return_gap = max(0, ann_mkt_mean - ann_strat_mean)
    return_penalty = 1 + (return_gap**2) / 100
    return sharpe / (vol_penalty * return_penalty)

# ============================================
# 1. 数据加载与特征工程
# ============================================
print("Step 1: Loading & Generating Features...")
train_all = pd.read_csv('/kaggle/input/hull-tactical-market-prediction/train.csv')
train_all = train_all.sort_values('date_id').reset_index(drop=True)
train_all['past_returns'] = train_all['forward_returns'].shift(1).fillna(0)

lags = [1, 2, 3, 5, 10, 21, 63]
for lag in lags:
    train_all[f'ret_lag_{lag}'] = train_all['past_returns'].shift(lag).fillna(0)

train_all['vol_20'] = train_all['past_returns'].rolling(20).std().fillna(0)
train_all['vol_60'] = train_all['past_returns'].rolling(60).std().fillna(0)
train_all['skew_60'] = train_all['past_returns'].rolling(60).skew().fillna(0)
train_all['mom_20'] = train_all['past_returns'].rolling(20).sum().fillna(0)
train_all['mom_60'] = train_all['past_returns'].rolling(60).sum().fillna(0)
train_all['mom_ratio'] = train_all['mom_20'] / (train_all['mom_60'].abs() + 1e-9)
train_all['vol_ratio'] = train_all['vol_20'] / (train_all['vol_60'] + 1e-9)
train_all['ret_vol_int'] = train_all['past_returns'] * train_all['vol_20']

safe_price = (1 + train_all['past_returns']).cumprod()
def rsi_wilder_safe(series, period=14):
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/period, min_periods=period).mean()
    avg_loss = loss.ewm(alpha=1/period, min_periods=period).mean()
    rs = avg_gain / (avg_loss + 1e-12)
    return 100 - 100 / (1 + rs)
train_all['rsi'] = rsi_wilder_safe(safe_price).fillna(50)

train_all['ma_long'] = train_all['past_returns'].rolling(window=200).mean().fillna(0)
train_all['ma_short'] = train_all['past_returns'].rolling(window=10).mean().fillna(0)
train_all['trend_mask'] = np.where(train_all['ma_short'] < train_all['ma_long'], 0.95, 1.0)

vol_200 = train_all['past_returns'].rolling(window=200).std()
vol_60 = train_all['past_returns'].rolling(window=60).std()
train_all['long_vol'] = np.sqrt(0.7 * vol_200**2 + 0.3 * vol_60**2).fillna(method='ffill').fillna(0.01)

# ============================================
# 2. 数据切分
# ============================================
print("Step 2: Splitting & Preprocessing...")
meta_cols = ['date_id', 'forward_returns', 'risk_free_rate',
             'market_forward_excess_returns', 'trend_mask', 'long_vol', 'vol_20']
feature_cols = [c for c in train_all.columns if c not in meta_cols
                and c not in ['ma_long', 'ma_short', 'past_returns', 'mom_20', 'mom_60']]

sim_test = train_all.iloc[-TEST_DAYS:].copy().reset_index(drop=True)
train = train_all.iloc[:-TEST_DAYS].copy().reset_index(drop=True)

X_train_raw = train[feature_cols].ffill().fillna(0)
X_test_raw = sim_test[feature_cols].ffill().fillna(0)

scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train_raw)
X_test_scaled = scaler.transform(X_test_raw)

if 'market_forward_excess_returns' in train.columns:
    y_train = train['market_forward_excess_returns']
else:
    y_train = train['forward_returns']
y_train_class = (y_train > 0).astype(int)

# ============================================
# 2.5 定义神经网络类
# ============================================
print("Step 2.5: Defining Neural Network Classes...")

class SharpeLoss(nn.Module):
    def __init__(self, ann_factor=252):
        super(SharpeLoss, self).__init__()
        self.ann_factor = ann_factor
    def forward(self, output, target_returns):
        strategy_returns = output * target_returns
        mean_ret = torch.mean(strategy_returns)
        std_ret = torch.std(strategy_returns) + 1e-6
        sharpe = (mean_ret / std_ret) * torch.sqrt(torch.tensor(self.ann_factor))
        return -sharpe

class SharpeMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, dropout=0.3):
        super(SharpeMLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.SiLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.SiLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, 1),
            nn.Tanh()
        )
    def forward(self, x): return self.net(x)

# ============================================
# 3. Walk-Forward OOF 生成
# ============================================
print("\n" + "="*60)
print("⏳ STARTING WALK-FORWARD VALIDATION")
print("="*60)

TEST_WINDOW = 180
INITIAL_TRAIN_YEARS = 5
MIN_TRAIN_DAYS = 252 * INITIAL_TRAIN_YEARS

oof_pred_ridge = np.full(len(train), np.nan)
oof_pred_nn = np.full(len(train), np.nan)
oof_pred_prob = np.full(len(train), np.nan)

X_full_raw = train[feature_cols].replace([np.inf, -np.inf], 0).ffill().fillna(0).values
y_full_target = train['market_forward_excess_returns'].values
y_full_raw = train['forward_returns'].values
y_full_cls = (y_full_target > 0).astype(int)

splits = range(MIN_TRAIN_DAYS, len(train), TEST_WINDOW)
print(f"🔄 Total Folds to Run: {len(splits)}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for i, split_idx in enumerate(tqdm(splits)):
    train_end = split_idx
    test_end = min(split_idx + TEST_WINDOW, len(train))
    if train_end >= len(train): break

    X_tr = X_full_raw[:train_end]
    y_tr_r = y_full_target[:train_end]
    y_tr_n = y_full_raw[:train_end]
    y_tr_c = y_full_cls[:train_end]
    X_te = X_full_raw[train_end:test_end]

    scaler_iter = RobustScaler()
    X_tr_sc = scaler_iter.fit_transform(X_tr)
    X_te_sc = scaler_iter.transform(X_te)

    # 1. Ridge
    model_r = Ridge(alpha=100)
    model_r.fit(X_tr_sc, y_tr_r)
    oof_pred_ridge[train_end:test_end] = model_r.predict(X_te_sc)

    # 2. LGBM
    model_l = lgb.LGBMClassifier(n_estimators=60, learning_rate=0.05, verbose=-1, random_state=42, n_jobs=1)
    model_l.fit(X_tr_sc, y_tr_c)
    oof_pred_prob[train_end:test_end] = model_l.predict_proba(X_te_sc)[:, 1]

    # 3. NN
    temp_model = SharpeMLP(input_dim=X_tr.shape[1]).to(device)
    optimizer = optim.Adam(temp_model.parameters(), lr=0.001)
    criterion = SharpeLoss()
    t_X_tr = torch.tensor(X_tr_sc, dtype=torch.float32).to(device)
    t_y_tr = torch.tensor(y_tr_n.reshape(-1,1), dtype=torch.float32).to(device)

    temp_model.train()
    for epoch in range(12):
        optimizer.zero_grad()
        out = temp_model(t_X_tr)
        loss = criterion(out, t_y_tr)
        loss.backward()
        optimizer.step()

    temp_model.eval()
    with torch.no_grad():
        t_X_te = torch.tensor(X_te_sc, dtype=torch.float32).to(device)
        pred_n = temp_model(t_X_te).cpu().numpy().flatten()
    oof_pred_nn[train_end:test_end] = pred_n

print("✅ Walk-Forward Generation Complete.")

# ============================================
# Step 5: Monte Carlo Robust Optimization
# ============================================
print("\n" + "="*60)
print("🧬 STEP 5: ROBUST PARAMETER SEARCH (MONTE CARLO + CVaR)")
print("="*60)

# --- 1. 准备有效数据 ---
valid_mask = ~np.isnan(oof_pred_ridge)
# 预测值
v_ridge = oof_pred_ridge[valid_mask]
v_nn = oof_pred_nn[valid_mask]
v_prob = oof_pred_prob[valid_mask]
# 市场数据
v_vol20 = train['vol_20'].values[valid_mask]
v_long = train['long_vol'].values[valid_mask]
v_trend = train['trend_mask'].values[valid_mask]
v_ret = train['forward_returns'].values[valid_mask]
v_rf = train['risk_free_rate'].values[valid_mask]
v_mkt = train['forward_returns'].values[valid_mask]

# 标准化 (OOF Stats)
R_MEAN, R_STD = np.mean(v_ridge), np.std(v_ridge)
N_MEAN, N_STD = np.mean(v_nn), np.std(v_nn)
rank_ridge = np.clip((v_ridge - R_MEAN) / (R_STD + 1e-9), -3, 3) / 3.0
rank_nn = np.clip((v_nn - N_MEAN) / (N_STD + 1e-9), -3, 3) / 3.0
confidence = np.abs((v_prob**2) - ((1-v_prob)**2))

# --- 2. 预生成 1000 个平行宇宙 ---
N_SIMS = 1000
SLICE_LEN = 180
max_idx = len(v_ret) - SLICE_LEN

np.random.seed(42)
start_idxs = np.random.randint(0, max_idx, N_SIMS)
indices = start_idxs[:, None] + np.arange(SLICE_LEN)

# 预切片
mc_vol20 = v_vol20[indices]
mc_long = v_long[indices]
mc_trend = v_trend[indices]
mc_ret = v_ret[indices]
mc_rf = v_rf[indices]
mc_mkt = v_mkt[indices]
# 信号数据
mc_ridge = rank_ridge[indices]
mc_nn = rank_nn[indices]
mc_conf = confidence[indices]

# 预计算固定杠杆部分
mc_alpha_scale = np.clip(TARGET_VOL / (mc_vol20 * np.sqrt(252) + 1e-6), 0.7, 1.3)
mc_raw_lev = TARGET_VOL / (mc_long * np.sqrt(252) + 1e-6)
mc_port_lev = np.clip(mc_raw_lev, 0.8, 1.25)
mc_combined_lev = mc_alpha_scale * mc_port_lev * mc_trend

# --- 3. 向量化评分函数 ---
def vectorized_hull_score(strat_ret_matrix, mkt_ret_matrix, rf_matrix):
    # 策略指标
    excess = strat_ret_matrix - rf_matrix
    mean_exc = np.mean(excess, axis=1)
    std_strat = np.std(strat_ret_matrix, axis=1) + 1e-9

    sharpe = (mean_exc / std_strat) * np.sqrt(252)
    vol_strat = std_strat * np.sqrt(252) * 100

    # 市场指标
    mkt_exc = mkt_ret_matrix - rf_matrix
    mean_mkt = np.mean(mkt_exc, axis=1)
    std_mkt = np.std(mkt_ret_matrix, axis=1) + 1e-9
    vol_mkt = std_mkt * np.sqrt(252) * 100

    # 罚分
    ratio = vol_strat / (vol_mkt + 1e-9)
    exc_vol = np.maximum(0, ratio - 1.2)
    exc_vol = np.where(vol_mkt < 1e-4, 0, exc_vol)
    vol_pen = 1 + exc_vol

    ann_ret_strat = mean_exc * 252 * 100
    ann_ret_mkt = mean_mkt * 252 * 100
    gap = np.maximum(0, ann_ret_mkt - ann_ret_strat)
    ret_pen = 1 + (gap**2) / 100

    return sharpe / (vol_pen * ret_pen)

# --- 4. 暴力网格搜索 (含 ES/CVaR) ---
best_robust_score = -999
final_w = 0.5
final_m = 1.0

print(f"🚀 Simulating {N_SIMS} universes per parameter set...")
# 打印表头增加 ES_05 (Expected Shortfall 5%)
print(f"{'Ridge_W':<8} {'Mult_M':<8} | {'Mean':<8} {'VaR_5%':<8} {'ES_5%':<8} {'Win%':<6} | {'Score'}")
print("-" * 75)

for w in np.arange(0.0, 1.01, 0.2):
    mc_hybrid = (w * mc_ridge + (1-w) * mc_nn) * mc_conf

    for m in np.arange(0.5, 3.0, 0.25):
        # 计算仓位
        base_alloc = mc_port_lev * mc_trend
        active_alloc = mc_hybrid * m * mc_combined_lev
        alloc = np.clip(base_alloc + active_alloc, MIN_INVESTMENT, MAX_INVESTMENT)

        # 计算收益与分数
        strat_ret = mc_rf * (1 - alloc) + alloc * mc_ret
        scores = vectorized_hull_score(strat_ret, mc_mkt, mc_rf)

        # --- 统计指标核心 ---
        mean_s = np.mean(scores)

        # 1. 计算 5% 分位数 (VaR)
        percentile_05 = np.percentile(scores, 5)

        # 2. 计算 5% 以下的均值 (Expected Shortfall / CVaR)
        # 逻辑：找出所有小于等于 5% 分位数的分数，求平均
        tail_scores = scores[scores <= percentile_05]
        worst_05_mean = np.mean(tail_scores) if len(tail_scores) > 0 else percentile_05

        win_r = np.mean(scores > 0)

        # --- 稳健性打分规则 ---
        # 规则升级：如果 5% 尾部均值太差，直接毙掉
        # 这比只看 VaR 更安全，防止“黑天鹅”

        if worst_05_mean < -2.0: # 如果尾部均值低于 -2.0，说明一旦出事就是大事
            robust_metric = -99
        elif percentile_05 < -1.5: # 传统的 VaR 检查
            robust_metric = -50
        else:
            robust_metric = mean_s

        if m % 1.0 == 0:
             print(f"{w:<8.1f} {m:<8.2f} | {mean_s:<8.4f} {percentile_05:<8.4f} {worst_05_mean:<8.4f} {win_r:<6.2%} | {robust_metric:.4f}")

        if robust_metric > best_robust_score:
            best_robust_score = robust_metric
            final_w = w
            final_m = m

print("-" * 75)
print(f"🏆 Best Robust Parameters (Optimized for Tail Risk):")
print(f"✅ Optimal Ridge Weight: {final_w:.2f}")
print(f"✅ Optimal Multiplier:   {final_m:.2f}")
print(f"📊 Robust Metric Score:  {best_robust_score:.4f}")

# 自动安全修正 (基于 ES 逻辑)
if final_m > 2.2:
    print("\n⚠️ 自动安全修正: 降低杠杆以构建安全缓冲")
    final_m = min(final_m, 2.2)
    print(f"🔒 Final Safe Multiplier: {final_m}")

print("="*60 + "\n")

# ============================================
# [重要] 6. 最终训练与预测 (Step 6)
# ============================================
print("\nStep 6: Final Prediction on Test Set...")

# ============================================
# 缺失的函数定义：local_score_check
# ============================================
def local_score_check(sol, sub):
    """
    在测试集上计算最终的官方分数
    """
    sol = sol.copy().reset_index(drop=True)
    sub = sub.copy().reset_index(drop=True)
    sol['position'] = sub['prediction']

    # 策略净值计算
    sol['strategy_returns'] = sol['risk_free_rate'] * (1 - sol['position']) + sol['position'] * sol['forward_returns']

    # 调用官方评分函数 (需确保 calc_official_metric 已定义)
    score = calc_official_metric(
        strat_ret=sol['strategy_returns'],
        mkt_ret=sol['forward_returns'],
        rf=sol['risk_free_rate']
    )

    # 辅助指标打印
    strat_std = sol['strategy_returns'].std()
    market_std = sol['forward_returns'].std()
    strat_vol = strat_std * np.sqrt(252) * 100
    market_vol = market_std * np.sqrt(252) * 100
    vol_penalty = 1 + max(0, strat_vol / market_vol - 1.2)

    print(f"\n--- 🏁 Final Performance Report ---")
    print(f"Strategy Vol: {strat_vol:.2f}% (Market: {market_vol:.2f}%)")
    print(f"Vol Penalty:  {vol_penalty:.4f}")
    return score

# 1. 全量训练 Ridge
model_ridge = Ridge(alpha=100)
model_ridge.fit(X_train_scaled, y_train)
pred_r_test = model_ridge.predict(X_test_scaled)

# 2. 全量训练 LGBM
model_clf = lgb.LGBMClassifier(n_estimators=100, learning_rate=0.05, verbose=-1, random_state=42)
model_clf.fit(X_train_scaled, y_train_class)
pred_p_test = model_clf.predict_proba(X_test_scaled)[:, 1]

# 3. 全量训练 NN
# 定义外部训练函数
def train_nn_final(X, y, epochs=25):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = SharpeMLP(input_dim=X.shape[1]).to(device)
    optimizer = optim.AdamW(model.parameters(), lr=1e-3)
    criterion = SharpeLoss()
    t_X = torch.tensor(X, dtype=torch.float32).to(device)
    t_y = torch.tensor(y.values.reshape(-1,1), dtype=torch.float32).to(device)
    model.train()
    for e in range(epochs):
        optimizer.zero_grad()
        out = model(t_X)
        loss = criterion(out, t_y)
        loss.backward()
        optimizer.step()
    return model, device

y_train_raw = train['forward_returns']
model_nn_final, device = train_nn_final(X_train_scaled, y_train_raw)

t_X_test = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
with torch.no_grad():
    pred_n_test = model_nn_final(t_X_test).cpu().numpy().flatten()

# 4. 混合与生成结果
z_r = np.clip((pred_r_test - R_MEAN) / (R_STD + 1e-9), -3, 3) / 3.0
z_n = np.clip((pred_n_test - N_MEAN) / (N_STD + 1e-9), -3, 3) / 3.0
conf = np.abs((pred_p_test**2) - ((1-pred_p_test)**2))

y_final = (final_w * z_r + (1-final_w) * z_n) * conf

curr_vol = sim_test['vol_20'].ffill().fillna(0.01).values
alpha_s = np.clip(TARGET_VOL / (curr_vol * np.sqrt(252) + 1e-6), 0.7, 1.3)
curr_long = sim_test['long_vol'].values
port_l = np.clip(TARGET_VOL / (curr_long * np.sqrt(252) + 1e-6), 0.8, 1.25)
trend = sim_test['trend_mask'].values

alloc_final = np.clip((1.0 + y_final * final_m * alpha_s) * port_l * trend, MIN_INVESTMENT, MAX_INVESTMENT)

submission = pd.DataFrame({'date_id': sim_test['date_id'], 'prediction': alloc_final})

# ============================================
# 7. 评分检查
# ============================================
final_score = local_score_check(sim_test, submission)
print(f"✅ Final Test Score: {final_score:.4f}")